# Data Analysis - Exercise 3
This lecture is about basics of the time series forecasting. We will discuss the natural gas consumption forecasting topic using provided dataset.

Raw dataset is available at [vsb.ai](https://vsb.ai/natural-gas-forecasting), but we will use already pre-processed version of it. You can download the simplified version from our [Github](https://github.com/rasvob/2020-2021-DA4).

[Open in Google colab](https://colab.research.google.com/github/rasvob/2020-2021-DA4/blob/master/03_Forecasting.ipynb)

[Download from Github](https://github.com/rasvob/2020-2021-DA4/blob/master/03_Forecasting.ipynb)

## We start with importing commonly used libraries. 
Nothing new here, I assume that you already know most of them.
## We will use maily Plotly (you can see [this link](https://plotly.com/python/plotly-express/) for more information) for our visualizations.
Plotly is a nice alternative to Matplotlib or Seaborn as it uses Javascript-backed plots with very straightforward API and basic interactivity out of the box.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# from plotly.offline import init_notebook_mode
# init_notebook_mode(connected=False)
pd.set_option('display.max_colwidth', 100)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
!pip uninstall -y statsmodels
!pip install statsmodels==0.12.2

In [ ]:
from statsmodels.tsa.stattools import acf, pacf, ccf, ccovf
from statsmodels.tsa.seasonal import STL
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf 

We need to install the rstl dependency from pip for the time series decomposition.

In [ ]:
!pip install rstl
import rstl

In [ ]:
!pip install tqdm
from tqdm.notebook import trange, tqdm
tqdm.pandas()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima.model import ARIMA 

We have prepared common metrics for model evaluation beforehand. We will use these functions later.

In [ ]:
"""
Computes MAPE
"""
def mean_absolute_percentage_error(y_true: np.array, y_pred: np.array) -> float:
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

"""
Computes SMAPE
"""
def symetric_mean_absolute_percentage_error(y_true: np.array, y_pred: np.array) -> float:
    return np.mean(np.abs((y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2.0))) * 100

"""
Computes MAE, MSE, MAPE, SMAPE, R2
"""
def compute_metrics(df: pd.DataFrame) -> pd.DataFrame:
    y_true, y_pred = df['y_true'].values, df['y_pred'].values
    return compute_metrics_raw(y_true, y_pred)

def compute_metrics_raw(y_true: pd.Series, y_pred: pd.Series) -> pd.DataFrame:
    mae, mse, mape, smape, r2 = mean_absolute_error(y_true=y_true, y_pred=y_pred), mean_squared_error(y_true=y_true, y_pred=y_pred), mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred), symetric_mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred), r2_score(y_true=y_true, y_pred=y_pred)
    return pd.DataFrame.from_records([{'MAE': mae, 'MSE': mse, 'MAPE': mape, 'SMAPE': smape, 'R2': r2}], index=[0])

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/rasvob/2020-2021-DA4/master/datasets/ppnet_metar_v8_MAD.csv', sep=';', index_col=0)

### We will drop year 2019 for now and use only years 2013 to 2018

In [ ]:
df = df[df.Year < 2019].copy()

In [ ]:
df.loc[:, 'TestSet'] = 0
df.loc[df.Year == 2018, 'TestSet'] = 1

Dataset covers six whole years from January 1, 2013 to December 31, 2018. All data features are available at an hourly frequency. The whole dataset is composed of 52,584 data points. These data points were assembled from three main components.

The first component was created from consumption data. Prague is the capital city of the Czech Republic and its distribution network consisted of 422,926 customers in 2018. Total consumption was 3.82 billion m3. Residential sector included 381,914 households (33.3% of consumption). Industrial sector consisted from 177 big (24.8% of consumption), 39,175 medium (18.9% of consumption) and 1,652 small customers (21.9% of consumption). Missing remainder to 100% were operational losses that occurred during distribution, e.g., pipeline leak. The heating season in the Czech Republic is from September 1 to May 31. Usually, it is required for the heating season to begin that the temperature drops below +13 °C for two consecutive days, and no warming is forecasted for the following days. The heating season usually represents about 70% - 75% of the whole year's consumption.

The second component includes weather variables. We have used data from the Prague LKPR airport weather station. Airports are required to periodically issue METAR (aerodrome routine meteorological report) information. Those reports are archived and preserved for a long time.

The third component representing economic features are natural gas price data. We have obtained price data from the Czech energy regulation office and included them in the dataset.

In [ ]:
df.head()

### We have multiple features in the dataset. Consumption is the forecasted endogenous variable, other features are treated as exogenous.

In [ ]:
df.columns

We have circa 52k datapoints which should be sufficient even for very complex models.

In [ ]:
df.shape

## Take a look at the viz' below. Do you see any patterns in the data?
- Note: You can zoom to see more details.

In [ ]:
px.line(y=df['Consumption'], x=df.index, color=df.Year)

In [ ]:
px.line(y=df['Consumption'], x=df.index, color=df.TestSet)

## What about now? Can we make any assumptions about the data?

In [ ]:
fig = px.box(df, y='Consumption', color='Month', facet_row='TestSet')
fig.update_layout(
    height=1000
)

### We have no missing values. Can you think about some easy ways to deal with the missing data if we had the issue?

In [ ]:
df.isna().sum()

### We need to split the data into two parts. X is the model input and y is the output - gas consumption.

In [ ]:
X, y = df.loc[:, df.columns != 'Consumption'].copy(), df.Consumption.copy()

### It is very important to check if the time series is self-correlated. We will use auto-correlation function for this task which computes correlation between raw values and lagged ones.

### What patterns do you see in the data?

In [ ]:
res_acf = acf(y, nlags=172)
fig = go.Figure()
fig.add_trace(go.Scatter(x=[i for i in range(0, len(res_acf))], y=res_acf, mode='markers'))
fig.show()

### We can take a look at the dependency of the consumption variable on the other exogenous factors as well. We will use cross-correlation function for this purpose.
#### What seems important to you based on the plots?

In [ ]:
res_ccf = ccf(x=X.Temperature, y=y, unbiased=True)
res_ccf.shape
fig = go.Figure()
fig.add_trace(go.Scatter(x=[i for i in range(0, len(res_ccf[:168*2]))], y=res_ccf[:168*2], mode='markers'))
fig.show()

In [ ]:
res_ccf = ccf(x=X.Humidity, y=y, unbiased=True)
res_ccf.shape
fig = go.Figure()
fig.add_trace(go.Scatter(x=[i for i in range(0, len(res_ccf[:168*2]))], y=res_ccf[:168*2], mode='markers'))
fig.show()

In [ ]:
res_ccf = ccf(x=X.Wind_speed, y=y, unbiased=True)
res_ccf.shape
fig = go.Figure()
fig.add_trace(go.Scatter(x=[i for i in range(0, len(res_ccf[:168*2]))], y=res_ccf[:168*2], mode='markers'))
fig.show()

# We will use 24h long forecast horizon. This length will be utilized in other examples as well.

## We will start with something simple.
You probably remember simple autoregressive model (AR) from the monday lecture. We will start with AR(24) model, which means that the forecasted value will be a linear combination of the 24 previous values.

In [ ]:
y_train = y[X.TestSet == 0].dropna()

In [ ]:
y_test = y[X.TestSet == 1].dropna()

### We will fit the model on training data at first.

In [ ]:
model = ARIMA(y_train.asfreq('H'),order=(24, 0, 0), trend='n', freq='H', enforce_stationarity=True, enforce_invertibility=True)
res = model.fit()
res = res.append(endog=y_test[0:1].asfreq('H'))

In [ ]:
y_pred = res.forecast(24)
forecasts = y_pred.copy()

### Then we need to feed the true consumption values for the last 24 hours into the model.

In [ ]:
for i in range(0, 365):
    if 1 + (i+1)*24 > len(y_test):
        break
    res = res.extend(endog=y_test[1 + i*24:1 + (i+1)*24].asfreq('H'))
    y_pred = res.forecast(24)
    forecasts = forecasts.append(y_pred)

# This is just for alignment of the true and forecasted time series
y_pred = forecasts[:-1]
y_test = y_test[1:].copy()

### Now we can compute the model error metrics and vizualize the results.

In [ ]:
df_res = pd.DataFrame({'y_true': y_test, 'y_pred': y_pred}, index=y_test.index)
df_res.head()

### We were able to obtain relatively good results even with model that simple.

In [ ]:
compute_metrics(df_res)

In [ ]:
df_res_s = df_res.stack().reset_index().rename({'level_1': 'Type', 0: 'Value'}, axis=1)
px.line(df_res_s, y='Value', x='Datetime', color='Type')

# The AR model utilizes only a single variable, it is an univariate model after all, but we have more covariate time series available.
### We will now move to more complex models based on machine learning algorithms.
### We will utilize only a single model for the all 24 forecasts and we will do the forecasting in the direct manner, i.e. no interdependency among multiple forecasted values. 

# Dataset needs to be built for the ML model with lagged variable values as features.
### You are free to use any variables you want. We will demonstrate the usage only with a subset of them.

In [ ]:
X, y = df.loc[:, df.columns != 'Consumption'].copy(), df.Consumption.copy()

In [ ]:
for x in trange(24, 169):
    X.loc[:, f'Consumption_lag_{x}'] = y.shift(x)

In [ ]:
for x in trange(24, 24*2+1):
    X.loc[:, f'Temperature_lag_{x}'] = X['Temperature'].shift(x)
    X.loc[:, f'Humidity_lag_{x}'] = X['Humidity'].shift(x)
    X.loc[:, f'Cena_lag_{x}'] = X['Cena_bfill'].shift(x)

In [ ]:
for x in trange(167, 169):
    X.loc[:, f'Temperature_lag_{x}'] = X.loc[:, 'Temperature'].shift(x)
    X.loc[:, f'Humidity_lag_{x}'] = X.loc[:, 'Humidity'].shift(x)
    X.loc[:, f'Cena_lag_{x}'] = X.loc[:, 'Cena_bfill'].shift(x)

## Following technique is called cyclical-encoding. 
Imagine that you have days from Monday to Sunday encoded with numbers from 1 to 7. Tuesday is one day after Monday, thus 2 is after 1 and 2 - 1 = 1 day difference. This holds through whole week except for Sunday, because 1 does not go after the 7 according to the used encoding scheme. We can deal with the situation with mapping days of the week (and other time related information) into 2D space using goniometric functions. These functions are periodic and distance of Sunday and Monday is the same as in the other cases.

In [ ]:
X.loc[:, 'Day_of_week_sin'] = np.sin(2 * np.pi * X['Day_of_week']/X['Day_of_week'].max())
X.loc[:, 'Day_of_week_cos'] = np.cos(2 * np.pi * X['Day_of_week']/X['Day_of_week'].max())

In [ ]:
X.loc[:, 'Month_sin'] = np.sin(2 * np.pi * X['Month']/X['Month'].max())
X.loc[:, 'Month_cos'] = np.cos(2 * np.pi * X['Month']/X['Month'].max())

In [ ]:
X.loc[:, 'Hour_sin'] = np.sin(2 * np.pi * X['Hour']/X['Hour'].max())
X.loc[:, 'Hour_cos'] = np.cos(2 * np.pi * X['Hour']/X['Hour'].max())

In [ ]:
X.loc[:, 'Day_sin'] = np.sin(2 * np.pi * X['Day']/X['Day'].max())
X.loc[:, 'Day_cos'] = np.cos(2 * np.pi * X['Day']/X['Day'].max())

# We can feature engineer even more exogenous variables. We can for example mark certain months as a summer season explicitly.
- We saw in the beggining of the lecture that the consumption is very low during the summer compared to the other seasons.

In [ ]:
X['IsSummer'] = 0
X.loc[X.Month.between(6, 8), 'IsSummer'] = 1

## You can imagine that people turn on the heating in their homes based on the month and temperature. We can mark the probable heating season with the so-called dummy variable.
- This is simplified variant of a heating season definition, there are multiple ways how to do this.
- We can mark weekend period as well.

In [ ]:
X['IsHeatingSeason'] = 1
X.loc[X.Month.between(6, 8), 'IsHeatingSeason'] = 0
heat_final = X.apply(lambda x: 1 if x['Temperature'] < 13 and x['IsHeatingSeason'] == 1 else 0, axis=1)
X['IsHeatingSeason'] = heat_final

In [ ]:
X['IsWeekend'] = 0
X.loc[X.Day_of_week.between(6, 7), 'IsWeekend'] = 1

## We can't use one model for multiple outputs in case of a tree-based algorithm. 
#### We will use the methodology we defined in our paper [Short-term natural gas consumption forecasting from long-term data collection](https://doi.org/10.1016/j.energy.2020.119430) which uses the difference from a fixed point in time as an output of the forecasting model.
#### Midnight is this fixed point in our case. So every forecasted value will be treated difference as a difference from the last midnight.

First mark the midnights rows.

In [ ]:
X['ResetSignal'] = 0
X.loc[X.Hour == 0, 'ResetSignal'] = 1

Define the new column in our dataset.

In [ ]:
X['Residual_diff_from_midnight'] = 0

In [ ]:
X['Residual'] = y

In [ ]:
midnight = X.Residual[0]
col_idx = X.columns.get_loc('Residual_diff_from_midnight')
for row_idx in trange(1, X.shape[0]):
    row = X.iloc[row_idx]
    val = row.Residual - midnight
    X.iloc[row_idx, col_idx] = val
    if row.ResetSignal == 1:
        midnight = row.Residual

In [ ]:
px.line(x=X.index, y=X['Residual_diff_from_midnight'])

In [ ]:
for x in range(24, 24*2+1):
    X.loc[:, f'Residual_diff_from_midnight_lag_{x}'] = X['Residual_diff_from_midnight'].shift(x)
    
for x in range(167, 169):
    X.loc[:, f'Residual_diff_from_midnight_lag_{x}'] = X['Residual_diff_from_midnight'].shift(x)

# Now we can split the data into train and test set

In [ ]:
y = X.Residual_diff_from_midnight

In [ ]:
X_train, X_test, y_train, y_test = X[X.TestSet == 0], X[X.TestSet == 1], y[X.TestSet == 0], y[X.TestSet == 1]

# We need to drop certain features so we don't have information leak in our model. 
- E.g. current temperature
- Some rows need to be dropped as well - we don't know previous consumption values for the first 24 rows in the dataset.

In [ ]:
X_train_selected_features_nona = X_train.drop(['Residual_diff_from_midnight', 'Year', 'Month', 'Day', 'Hour', 'Day_of_week','Temperature', 'Pressure', 'Pressure2', 'Humidity', 'Wind_direction', 'Wind_speed', 'Phenomena', 'Recent_phenomena', 'Visibility', 'Dewpoint', 'Datetime.1', 'Clouds_low_text', 'Clouds_low_m', 'Clouds_medium_text', 'Clouds_medium_m', 'Clouds_high_text', 'Clouds_high_m', 'Residual', 'ResetSignal', 'IsMissing', 'Cena_bfill', 'TestSet'], axis=1).dropna()
X_test_selected_features_nona = X_test.drop(['Residual_diff_from_midnight', 'Year', 'Month', 'Day', 'Hour', 'Day_of_week','Temperature', 'Pressure', 'Pressure2', 'Humidity', 'Wind_direction', 'Wind_speed', 'Phenomena', 'Recent_phenomena', 'Visibility', 'Dewpoint', 'Datetime.1', 'Clouds_low_text', 'Clouds_low_m', 'Clouds_medium_text', 'Clouds_medium_m', 'Clouds_high_text', 'Clouds_high_m', 'Residual', 'ResetSignal', 'IsMissing', 'Cena_bfill', 'TestSet'], axis=1).dropna()
y_train_no_na = y_train.dropna()
y_test_no_na = y_test.dropna()
y_train_no_na = y_train_no_na[y_train_no_na.index.isin(X_train_selected_features_nona.index)]
X_test_selected_features_nona = X_test_selected_features_nona[X_test_selected_features_nona.index.isin(y_test_no_na.index)]
X_train_selected_features_nona.shape, y_train_no_na.shape, X_test_selected_features_nona.shape, y_test_no_na.shape

# Dataset is ready so we can train the model now. We will use Light-gradient boosting algorithm as our model. You can choose any ML algorithm you want, e.g. Random forest.

In [ ]:
alg = LGBMRegressor(n_estimators=200, n_jobs=4, random_state=13)

In [ ]:
alg.fit(X_train_selected_features_nona, y_train_no_na)

In [ ]:
y_pred = alg.predict(X_test_selected_features_nona)

## We can take a look at the forecasted values by the model. 

In [ ]:
df_res = pd.DataFrame({'y_true': y_test_no_na.values, 'y_pred': y_pred}, index=y_test_no_na.index)

In [ ]:
df_res

In [ ]:
df_res_s = df_res.stack().reset_index().rename({'level_1': 'Type', 0: 'Value'}, axis=1)

In [ ]:
px.line(df_res_s, y='Value', x='Datetime', color='Type')

In [ ]:
compute_metrics(df_res)

## We can see that lagged values which were day/week before are the most valuable as well as sin/cos encoded time features and our engineered dummy variables.

In [ ]:
df_feat_imp = pd.DataFrame({'FeatureName': X_train_selected_features_nona.columns, 'FeatureImportance': alg.feature_importances_}).sort_values(by='FeatureImportance', ascending=False)
px.bar(df_feat_imp.sort_values(by='FeatureImportance', ascending=False).iloc[:15, :], y='FeatureName', x='FeatureImportance', orientation='h')

## Okay, we have forecasted values but what now?
Remember that we are forecasting the difference from last midnight. The next step is reconstruction of the data, so we obtain raw natural gas consumption.

In [ ]:
y_test_midnight = X[X.TestSet == 1].apply(lambda x: x['Residual'] if x['Hour'] == 0 else np.nan, axis=1).ffill().shift(1)
y_test_midnight['2018-01-01 00:00:00'] = X[X.index == '2017-12-31 00:00:00'].Residual
y_test_midnight

In [ ]:
ps_y_pred = pd.Series(y_pred, index=y_test_no_na.index)

In [ ]:
ps_y_pred = ps_y_pred + y_test_midnight

In [ ]:
ps_y_pred

In [ ]:
orig_data = df[df.TestSet == 1].Consumption
orig_data.index = pd.DatetimeIndex(orig_data.index)

In [ ]:
df_res = df_res = pd.DataFrame({'y_true': orig_data, 'y_pred': ps_y_pred})

In [ ]:
df_res

## We can compute the error measurement metrics and vizualize our results in this phase.

In [ ]:
compute_metrics(df_res.dropna())

In [ ]:
df_res_s = df_res.stack().reset_index().rename({'level_1': 'Type', 0: 'Value'}, axis=1)

In [ ]:
px.line(df_res_s, y='Value', x='Datetime', color='Type')

## What can we assume about the error distribution?

In [ ]:
ps_err = df_res.apply(lambda x: x[0] - x[1], axis=1)

In [ ]:
px.line(x=ps_err.index, y=ps_err)

## If you are interested in the daily results, it is no problem to aggregate the results.

In [ ]:
df_res_daily = df_res.resample('D').sum()

In [ ]:
df_res_daily

In [ ]:
df_res_s_daily = df_res_daily.stack().reset_index().rename({'level_1': 'Type', 0: 'Value'}, axis=1)

In [ ]:
px.line(df_res_s_daily, y='Value', x='Datetime', color='Type')

In [ ]:
compute_metrics(df_res_daily.dropna())

# Now we can try another preprocessing techniques. The first one is called differencing.
### Do you remember how it works?

In [ ]:
X['Residual_diff_from_midnight'] = 0

### Why diff by 24 hours? Think about the ACF plot and forecast horizon.

In [ ]:
y_diff = df.Consumption.diff(24).dropna()
X['Residual'] = y_diff

# Now we will use the STL decomposition algorithms to create an additive decomposed model from the differenced time series

In [ ]:
stl_decomp = rstl.STL(y_diff, freq=24*7, robust=False, s_window='periodic')

In [ ]:
trend, seasonal, residual, weights = stl_decomp.trend, stl_decomp.seasonal, stl_decomp.remainder, stl_decomp.weights

In [ ]:
px.line(x=y_diff.index, y=trend)

In [ ]:
px.line(x=y_diff.index, y=seasonal)

In [ ]:
px.line(x=y_diff.index, y=residual)

# If we take a closer look at the trend part, we can see that there is no trend present in the data and this part is very similiar to the residual one. Moreover it is lower by a magnitude thus we can merge the parts into one.
- Just beware that if the data had e.g. close-to-linear trend present, merging it with the residual part is not a good idea. It is better to use some simple separate model, e.g. exponential smoothing, AR, etc., to forecast trend part individualy.

In [ ]:
residual_trend = residual + trend

In [ ]:
X.loc[:, f'Seasonal'] = pd.Series((np.concatenate([[np.nan]*24, seasonal])), index=X.index)

In [ ]:
X.loc[:, f'Residual'] = pd.Series((np.concatenate([[np.nan]*24, residual_trend])), index=X.index)

In [ ]:
X.loc[:, f'Trend'] = pd.Series((np.concatenate([[np.nan]*24,trend])), index=X.index)

In [ ]:
seasonal_dict = {k:X[(X.TestSet == 0) & (X.Month == k)].groupby(['Day_of_week', 'Hour']).Seasonal.mean().reset_index() for k in  X.Month.value_counts().index.values}

def fill_seasonal(month, day, hour, seasonal_dict):
    season = seasonal_dict[month]
    return season[(season.Day_of_week == day) & (season.Hour == hour)].Seasonal.values[0]

In [ ]:
X.loc[X.TestSet == 1, 'Seasonal'] = X[X.TestSet == 1].loc[:, ['Month', 'Day_of_week', 'Hour']].progress_apply(lambda x: fill_seasonal(x[0], x[1], x[2], seasonal_dict), axis=1)

In [ ]:
for x in trange(1, 169):
    X.loc[:, f'Seasonal_lag_{x}'] = X['Seasonal'].shift(x)

In [ ]:
for x in trange(24, 169):
    X.loc[:, f'Residual_lag_{x}'] = X['Residual'].shift(x)

In [ ]:
midnight = X.Residual.iloc[0]
col_idx = X.columns.get_loc('Residual_diff_from_midnight')
for row_idx in trange(1, X.shape[0]):
    row = X.iloc[row_idx]
    val = row.Residual - midnight
    X.iloc[row_idx, col_idx] = val
    if row.ResetSignal == 1:
        midnight = row.Residual

## Does seem the distribution different from the last time? How?

In [ ]:
px.line(x=X.index, y=X['Residual_diff_from_midnight'])

In [ ]:
for x in trange(24, 24*2+1):
    X.loc[:, f'Residual_diff_from_midnight_lag_{x}'] = X['Residual_diff_from_midnight'].shift(x)
    X.loc[:, f'Residual_lag_{x}'] = X['Residual'].shift(x)
    
for x in range(167, 169):
    X.loc[:, f'Residual_diff_from_midnight_lag_{x}'] = X['Residual_diff_from_midnight'].shift(x)
    X.loc[:, f'Residual_lag_{x}'] = X['Residual'].shift(x)

# Now we can split the data into train and test set

In [ ]:
y = X.Residual_diff_from_midnight

In [ ]:
X_train, X_test, y_train, y_test = X[X.TestSet == 0], X[X.TestSet == 1], y[X.TestSet == 0], y[X.TestSet == 1]

# We need to drop certain features so we don't have information leak in our model. 
- E.g. current temperature
- Some rows need to be dropped as well - we don't know previous consumption values for the first 24 rows in the dataset.

In [ ]:
X_train_selected_features_nona = X_train.drop(['Residual_diff_from_midnight', 'Year', 'Month', 'Day', 'Hour', 'Day_of_week','Temperature', 'Pressure', 'Pressure2', 'Humidity', 'Wind_direction', 'Wind_speed', 'Phenomena', 'Recent_phenomena', 'Visibility', 'Dewpoint', 'Datetime.1', 'Clouds_low_text', 'Clouds_low_m', 'Clouds_medium_text', 'Clouds_medium_m', 'Clouds_high_text', 'Clouds_high_m', 'Residual', 'ResetSignal', 'IsMissing', 'Cena_bfill', 'TestSet'], axis=1).dropna()
X_test_selected_features_nona = X_test.drop(['Residual_diff_from_midnight', 'Year', 'Month', 'Day', 'Hour', 'Day_of_week','Temperature', 'Pressure', 'Pressure2', 'Humidity', 'Wind_direction', 'Wind_speed', 'Phenomena', 'Recent_phenomena', 'Visibility', 'Dewpoint', 'Datetime.1', 'Clouds_low_text', 'Clouds_low_m', 'Clouds_medium_text', 'Clouds_medium_m', 'Clouds_high_text', 'Clouds_high_m', 'Residual', 'ResetSignal', 'IsMissing', 'Cena_bfill', 'TestSet'], axis=1).dropna()
y_train_no_na = y_train.dropna()
y_test_no_na = y_test.dropna()
y_train_no_na = y_train_no_na[y_train_no_na.index.isin(X_train_selected_features_nona.index)]
X_test_selected_features_nona = X_test_selected_features_nona[X_test_selected_features_nona.index.isin(y_test_no_na.index)]
X_train_selected_features_nona.shape, y_train_no_na.shape, X_test_selected_features_nona.shape, y_test_no_na.shape

In [ ]:
alg = LGBMRegressor(n_estimators=100, n_jobs=8, random_state=14)

In [ ]:
alg.fit(X_train_selected_features_nona, y_train_no_na)

In [ ]:
y_pred = alg.predict(X_test_selected_features_nona)

## We can take a look at the forecasted values by the model. 

In [ ]:
df_res = pd.DataFrame({'y_true': y_test_no_na.values, 'y_pred': y_pred}, index=y_test_no_na.index)

In [ ]:
df_res

In [ ]:
y_test_midnight = X[X.TestSet == 1].apply(lambda x: x['Residual'] if x['Hour'] == 0 else np.nan, axis=1).ffill().shift(1)
y_test_midnight['2018-01-01 00:00:00'] = X[X.index == '2017-12-31 00:00:00'].Residual
y_test_midnight

In [ ]:
ps_y_pred = pd.Series(y_pred, index=y_test_no_na.index)
ps_y_pred = ps_y_pred + y_test_midnight

In [ ]:
ps_shifted_residuals = (df.Consumption.shift(24)[(df.index >= '2018-01-01 00:00:00') & (df.index <= '2018-12-31 23:00:00')] + ps_y_pred) + X_test.Seasonal
ps_shifted_residuals.index = pd.DatetimeIndex(ps_y_pred.index)

In [ ]:
orig_data = df[df.TestSet == 1].Consumption
orig_data.index = pd.DatetimeIndex(orig_data.index)

In [ ]:
df_res = pd.DataFrame({'y_true': orig_data, 'y_pred': ps_shifted_residuals})

In [ ]:
df_res

## We can compute the error measurement metrics and vizualize our results in this phase.

In [ ]:
compute_metrics(df_res.dropna())

In [ ]:
df_res_s = df_res.stack().reset_index().rename({'level_1': 'Type', 0: 'Value'}, axis=1)

In [ ]:
px.line(df_res_s, y='Value', x='Datetime', color='Type')

# Finally we can imagine that the model could be re-trained during the year 2018 so it reflects some of the recent changes in the data.
- We can try to train 12 different models, one for each month.

In [ ]:
X_train_selected_features_nona.index = pd.DatetimeIndex(X_train_selected_features_nona.index)
X_test_selected_features_nona.index = pd.DatetimeIndex(X_test_selected_features_nona.index)
y_train_no_na.index = pd.DatetimeIndex(y_train_no_na.index)
y_test_no_na.index = pd.DatetimeIndex(y_test_no_na.index)
df.index = pd.DatetimeIndex(df.index)
df_res_final = pd.DataFrame({'y_true': df[df.TestSet == 1].Consumption, 'y_pred': np.nan}, index=pd.DatetimeIndex(y_test_no_na.index))
y_test_midnight = X[X.TestSet == 1].apply(lambda x: x['Residual'] if x['Hour'] == 0 else np.nan, axis=1).ffill().shift(1)
y_test_midnight['2018-01-01 00:00:00'] = X[X.index == '2017-12-31 00:00:00'].Residual
y_test_midnight.index = pd.DatetimeIndex(y_test_midnight.index)

In [ ]:
alg = LGBMRegressor(n_estimators=100, n_jobs=8, random_state=14)
alg.fit(X_train_selected_features_nona, y_train_no_na)
y_pred = alg.predict(X_test_selected_features_nona[X_test_selected_features_nona.index.month == 1])

In [ ]:
ps_y_pred = pd.Series(y_pred, index=y_test_no_na[y_test_no_na.index.month == 1].index)
ps_y_pred = ps_y_pred + y_test_midnight[y_test_midnight.index.month == 1]
ps_shifted_residuals = (df.Consumption.shift(24)[ps_y_pred.index] + ps_y_pred) + X_test_selected_features_nona.loc[ps_y_pred.index, 'Seasonal']
df_res_final.loc[ps_y_pred.index, 'y_pred'] = ps_shifted_residuals

In [ ]:
X_train_extended, y_train_extended = X_train_selected_features_nona.copy(), y_train_no_na.copy()
for x in trange(1, 12):
    X_train_extended = pd.concat([X_train_extended, X_test_selected_features_nona[X_test_selected_features_nona.index.month == x]])
    y_train_extended = pd.concat([y_train_extended, y_test_no_na[y_test_no_na.index.month == x]])
    next_month = x+1
    alg = LGBMRegressor(n_estimators=100, n_jobs=8, random_state=14)
    alg.fit(X_train_extended, y_train_extended)
    
    y_pred = alg.predict(X_test_selected_features_nona[X_test_selected_features_nona.index.month == next_month])
    ps_y_pred = pd.Series(y_pred, index=y_test_no_na[y_test_no_na.index.month == next_month].index)
    ps_y_pred = ps_y_pred + y_test_midnight[y_test_midnight.index.month == next_month]
    ps_shifted_residuals = (df.Consumption.shift(24)[ps_y_pred.index] + ps_y_pred) + X_test_selected_features_nona.loc[ps_y_pred.index, 'Seasonal']
    df_res_final.loc[ps_y_pred.index, 'y_pred'] = ps_shifted_residuals

In [ ]:
df_res_final

In [ ]:
compute_metrics(df_res_final.dropna())

In [ ]:
df_res_s = df_res_final.stack().reset_index().rename({'level_1': 'Type', 0: 'Value'}, axis=1)

In [ ]:
px.line(df_res_s, y='Value', x='Datetime', color='Type')

# Tasks for the rest of the lecture
1. Try to add more or remove lags of a selected exogenous variables and re-train the model with them.
2. Try different algorithms (Random forest or linear regression for example).
3. Try to re-train the model by longer or shorter period of time.
3. Compare the new model with the original one. Did the MAE, MSE etc changed? If it did, how?